
**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Edición 2021**

---

# Trabajo práctico entregable - Parte 2

In [1]:
import io
import matplotlib
import matplotlib.pyplot as plt
import numpy
import pandas as pd
import seaborn
from scipy import stats

seaborn.set_context('talk')

## Lectura del dataset

En la notebook 00 se explican los detalles de la siguiente sección.

In [2]:
url = 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/sysarmy_survey_2020_processed.csv'
df = pd.read_csv(url)

In [3]:
df[:3]

,profile_gender,profile_age,work_country,work_province,profile_years_experience,work_years_in_company,work_years_in_current_position,work_people_in_charge_of,profile_studies_level,profile_studies_level_state,...,work_has_violence_situations,profile_sexual_orientation,profile_has_disabilities,profile_has_disabilities_hiring_difficulties,company_employee_number,company_main_activity,company_recommended,company_diversity_policies,company_extra_benefits,company_best_companies_city
0,Mujer,26,Argentina,Ciudad Autónoma de Buenos Aires,3.0,3.0,3.0,0,Universitario,En curso,...,En mi trabajo actual,Homosexual,NaN,NaN,501-1000,Servicios / Consultoría de Software / Digital,7,2,"Capacitaciones y/o cursos, Comidas pagas / sub...",NaN
1,Hombre,29,Argentina,Corrientes,5.0,2.0,2.0,4,Universitario,En curso,...,Jamás,Heterosexual,Visual,No,201-500,Otras industrias,8,9,"Horarios flexibles, Stock options / RSUs, Viát...",NaN
2,Mujer,22,Argentina,Ciudad Autónoma de Buenos Aires,2.0,0.0,0.0,0,Secundario,Completado,...,En un trabajo anterior,Bisexual o queer,NaN,No,2001-5000,Otras industrias,6,9,"Clases de gimnasia online, Comidas pagas / sub...",NaN


In [4]:
df[['profile_gender', 'salary_monthly_NETO']].groupby('profile_gender').describe()

salary_monthly_NETO                                     \
                             count          mean            std   min   
profile_gender                                                          
Hombre                      4944.0  98836.063558  102159.664237   1.0   
Mujer                        908.0  73501.414465   56256.880181   2.0   
Otros                         31.0  95441.354839   88635.674358  40.0   

                                                       
                    25%      50%       75%        max  
profile_gender                                         
Hombre          53285.0  80000.0  110000.0  2080000.0  
Mujer           43818.0  65000.0   92000.0   800000.0  
Otros           39500.0  74000.0  117500.0   380000.0

In [5]:
df[df.salary_monthly_NETO > 1000]\
  [['profile_gender', 'salary_monthly_NETO']].groupby('profile_gender').describe()

salary_monthly_NETO                                         \
                             count           mean            std      min   
profile_gender                                                              
Hombre                      4815.0  101481.624343  102215.304684   1100.0   
Mujer                        863.0   77328.913778   55082.760570   2000.0   
Otros                         28.0  105660.714286   87235.710235  34000.0   

                                                       
                    25%      50%       75%        max  
profile_gender                                         
Hombre          55000.0  80000.0  110000.0  2080000.0  
Mujer           47850.0  67200.0   93000.0   800000.0  
Otros           43750.0  78900.0  122750.0   380000.0

In [6]:
alpha = 0.05

In [7]:
is_man = df.profile_gender == 'Hombre'

groupA = df[(df.salary_monthly_NETO > 1000) & is_man].salary_monthly_NETO
groupB = df[(df.salary_monthly_NETO > 1000) & ~is_man].salary_monthly_NETO

## Ejercicio 1: Estimación

**Consigna:**  Calcular una estimación puntual y un intervalo de confianza de nivel (1-alpha) para la resta entre la media del salario Neto para Hombres menos la media del salario Neto para otros géneros(diferencia de las medias entre el grupoA y grupoB). 
¿Cómo se relaciona este intervalo de confianza con el test de hipótesis?

La estimacion puntual se realiza mediante un estadistico que es un estimador del parametro que nos interesa. Lo que interesa en este punto es el salario neto promedio de dos grupos, uno identificado como Hombres y otro como No Hombres, por lo tanto, tendremos dos muestras de ambos grupos. Dado que no sabemos la media poblacion, se selecciona un estadistico (variable aleatoria que depende de la muestra) para realizar nuestra estimacion: la media muestral. Siguiendo la Ley de Los Grandes Numeros sabemos que la media muestral es un buen estimador de $\mu$ ya que mientras mayor sea el numero de nuestra muestra, la media muestral se acercara a la media poblacional (asumiendo que nuestras muestras tienen una distribucion normal). 

In [8]:
# Estimacion puntual: media muestral. 
u_groupA = groupA.mean()
u_groupB = groupB.mean()

print("Media Muestral GrupoA (hombres): ", u_groupA)
print("Media Muestral GrupoB (no-hombres): ", u_groupB)

Media Muestral GrupoA (hombres):  101481.62434267912
Media Muestral GrupoB (no-hombres):  78219.25094276095


La estimacion por intervalos consiste en obtener un intervalo de valores posibles para el parametro desconocido, de tal manera que con una cierta probabilidad, el intervalo contenga al verdadero parametro. 

En este caso tenemos que con un 95% de confianza, el verdadero parametro se encuentra entre los valores 101389 y 101573 para el grupo de hombres y entre un 78100 y 78338 para el grupo de no-hombres. 

El intervalo de confianza nos permite realizar el Metodo del Pivote, el cual, a partir de una distribucion conocida, nos permite obtener las zonas de Rechazo y No Rechazo del test de hipotesis. 

In [9]:
# Estimacion por intervalos (con 95% de confianza)

# GroupA = Hombres
ci_groupA = stats.t.interval(alpha, len(groupA), loc= u_groupA, scale=stats.sem(groupA)) # ci es el intervalo de confianza
print("Intervarlo de Confianza para Media Muestral (hombres): ", ci_groupA)

# GroupB - No Hombres
ci_groupB = stats.t.interval(alpha, len(groupB), loc= u_groupB, scale=stats.sem(groupB)) # ci es el intervalo de confianza
print("Intervarlo de Confianza para Media Muestral (no-hombres): ", ci_groupB)

Intervarlo de Confianza para Media Muestral (hombres):  (101389.24924608349, 101573.99943927475)
Intervarlo de Confianza para Media Muestral (no-hombres):  (78100.49257592262, 78338.00930959928)


## Ejercicio 2: Test de hipótesis



### 2.1 Formalización

Describir formalmente los distintos compenentes de un test de hipótesis para comprobar si la distribución de los salarios es distinta entre los grupos A y B.

**Hipótesis Nula**

La Hipotesis Nula significa que la diferencia de salario neto promedio entre ambos grupos es igual a 0, lo que significa que la diferencia que efectivamente observamos previamente no es significativa. Por ende, nuestra Hipotesis Alternativa indicaria que esta diferencia si es significativa y la media del grupo de personas identificadas como hombres es distinta de la media del grupo identificada como no-hombres. 

$H_0 : \mu_A - \mu_B = 0$  
$H_1 : \mu_A - \mu_B \neq 0$  


**Estadístico (Pivote)**
  * Identificar el estadístico
  * Escribir qué distribución tiene bajo $H_0$

1. El Estadistico que se utilizara sera la media muestral. 
2. La distribucion que tiene el estadistico bajo hipotesis nula es normal, ya que bajo este supuesto, podemos reconocer a la media muestral como un buen estadistico del parametro desconocido media de la poblacion, esto por la Ley de los Grandes Numeros. 

### 2.2 P-valor

1. Calcule el p-valor y decida si rechazar o no la hipótesis nula.
2. Interprete el resultado.
3. Los dos grupos de nuestra muestra tienen tamaños muy distintos. ¿Esto afecta al tests?

Links útiles:
* [Test de hipótesis usando scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)
* [Test de Welch](http://daniellakens.blogspot.com/2015/01/always-use-welchs-t-test-instead-of.html)

Si el tamaño del grupo A y B son iguales, el test - t es muy robusto contra varianzas distintas pero si el tamaño es distinta, las varianzas desiguales pueden afectar al error de tipo 1 del test t. Por lo tanto, antes tamaños de muestra tan disimiles, es conveniente utilizar Welch´s Test por default ya que, en el caso en que el tamaño de la muestra y la varianza son distintos entre grupos, tiene una mejor performance que el test t. Y en el caso de que sean iguales, da el mismo resultado. 

In [10]:
desv_groupA = numpy.sqrt(groupA.var())
desv_groupB = numpy.sqrt(groupB.var())

n_groupA = len(groupA)
n_groupB = len(groupB)

print("N Grupo Hombres: ", n_groupA)
print("N Grupo No-Hombres: ", n_groupB)

N Grupo Hombres:  4815
N Grupo No-Hombres:  891


In [11]:
test = stats.ttest_ind(groupA, groupB, equal_var = False)

print("Estadistico: ", test[0])
print("p-value: ", test[1])

Estadistico:  9.697205976350673
p-value:  8.605197322133209e-22


El p-valor obtenido en el test nos indica que existe evidencia para rechazar la Hipotesis Nula de igualdad de medias en ambos grupos, por lo tanto, estariamos en condiciones de decir que no rechazariamos la hipotesis alternativa que sugiere que la medias entre ambos grupos son distintas (no podemos saber si una es mayor que la otra, solo que son distintas).

### [Opcional] 2.3 Potencia del test

Nuestra muestra, ¿era lo suficientemente grande para detectar si existe o no una diferencia entre los grupos?

1. Utilice la función `tt_ind_solve_power` para calcular el tamaño necesario de la muestra para un poder estadístico de 0.8, 0.9 y 0.95, asumiendo una significancia estadística de 0.05.
2. ¿Cómo intepretan el poder estadístico de un test? Dado su conocimiento de dominio sobre los datos, ¿les parece que esta muestra es lo suficientemente grande para ser representativo de la tendencia general? ¿y para utilizarlo en un juicio penal contra una empresa XX por una causa de discriminación?

[Documentación](https://www.statsmodels.org/stable/generated/statsmodels.stats.power.tt_ind_solve_power.html)

NOTA: este análisis debería hacerse ANTES de recolectar los datos.

La potencia de un test, o poder estadistico, es la probabilidad de que la prueba de hipotesis rechace correctamente la hipotesis nula. Si una prueba tiene poca potencia, es probable que los resultados que hayamos obtenido no sean realmente significativos. Por lo tanto, es util llevar a cabo este test previo a realizar la prueba de hipotesis para saber si vale la pena trabajar con una muestra del tamaño Na = 4815 y Nb = 891. 

El resultado del test es el numero de observaciones de la muestra del grupo A requerido para satisfacer un determinado nivel de poder estadistico, el numero de la muestra del grupo B es ratio veces el tamaño de la muestra 1. Para cada nivel de potencia expresada en el enunciado se imprime el tamaño requerido para cada una de las muestras que permite obtener si la muestra es lo suficientemente grande para detectar si existe o no una diferencia significativa entre ambos grupos.

In [12]:
from statsmodels.stats.power import tt_ind_solve_power

In [13]:
effect_size = (groupA.mean() - groupB.mean()) / groupB.std()
nobs1=None  # What we want to know
alpha = 0.05
ratio = len(groupB) / len(groupA)
power = [0.8, 0.9, 0.95]

In [14]:
result_test = []

for i in power: 
    result_test.append(tt_ind_solve_power(effect_size=effect_size, nobs1 = nobs1, alpha=alpha, power=i, ratio=ratio))

In [15]:
print("POWER 0.8 -> N_groupA: {} - N_groupB: {}".format(int(result_test[0]), int(result_test[0]*ratio)))
print("POWER 0.9 -> N_groupA: {} - N_groupB: {}".format(int(result_test[1]), int(result_test[1]*ratio)))
print("POWER 0.95 -> N_groupA: {} - N_groupB: {}".format(int(result_test[2]), int(result_test[2]*ratio)))

POWER 0.8 -> N_groupA: 298 - N_groupB: 55
POWER 0.9 -> N_groupA: 398 - N_groupB: 73
POWER 0.95 -> N_groupA: 492 - N_groupB: 91


Los tamaños de muestra del grupo A y B con que contamos es bastante mayor que los obtenidos del test, por lo que podriamos realizar la prueba de hipotesis y obtener una diferencia entre ambas medias que es significativa. Sin embargo, una vez que contamos con ese resultado es necesario indagar si esa diferencia en el salario es por un motivo justificado, tal como mayor nivel educativo y/o experiencia, o si se justifica en motivos de indole discriminatorios. 